In [0]:
CREATE OR REPLACE TEMP VIEW country_wise_sales_temp_v AS
SELECT c.region, c.country, SUM(total_profit) / NULLIF(SUM(total_revenue), 0) * 100 AS profit_percentage, COUNT(DISTINCT order_id) AS country_order_count, current_timestamp() as load_timestamp, current_user() as loaded_by
FROM sales_fact_slvr s  
JOIN country_dim_slvr c  
ON c.id = s.country_id
GROUP BY c.region, c.country
ORDER BY profit_percentage DESC

In [0]:
MERGE INTO country_wise_sales_gold t
USING country_wise_sales_temp_v s
ON t.country = s.country
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

In [0]:
CREATE OR REPLACE TEMP VIEW week_wise_sales_temp_v AS
SELECT
  c.country,
  date_format(order_date, 'EEEE') AS order_weekday,
  COUNT(DISTINCT order_id) AS weekday_order_count,
  current_timestamp() as load_timestamp, current_user() as loaded_by
FROM sales_fact_slvr s  
JOIN country_dim_slvr c  
ON c.id = s.country_id
GROUP BY c.country, date_format(order_date, 'EEEE')
ORDER BY weekday_order_count DESC;

In [0]:
MERGE INTO week_wise_sales_gold t
USING week_wise_sales_temp_v s
ON t.country = s.country AND t.order_weekday = s.order_weekday
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

In [0]:
CREATE OR REPLACE TEMP VIEW order_fulfillment_temp_v AS
SELECT
  c.country,
  o.sales_channel,
  CASE o.order_priority
    WHEN 'L' THEN 'LOW'
    WHEN 'M' THEN 'MEDIUM'
    WHEN 'H' THEN 'HIGH'
    WHEN 'C' THEN 'CRITICAL'
  END AS order_priority,
  AVG(datediff(ship_date, order_date)) AS avg_fulfillment_days,
  PERCENTILE(datediff(ship_date, order_date), 0.5) AS median_fulfillment_days,
  PERCENTILE(datediff(ship_date, order_date), 0.9) AS p90_fulfillment_days,
  SUM(CASE WHEN datediff(ship_date, order_date) <= 3 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) 
      AS sla_3day_compliance,
   current_timestamp() as load_timestamp, current_user() as loaded_by
FROM sales_fact_slvr s  
JOIN country_dim_slvr c  
ON c.id = s.country_id
JOIN order_type_dim_slvr o  
ON o.id = s.order_type_id
GROUP BY c.country,
  o.sales_channel,
  o.order_priority
ORDER BY sla_3day_compliance DESC;


In [0]:
MERGE INTO order_fulfillment_gold t
USING order_fulfillment_temp_v s
ON t.country = s.country and t.sales_channel = s.sales_channel and t.order_priority = s.order_priority
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *